### Week 3 Capstone -Part 3

### PART 1 

In [9]:
#Import Library
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np

print("Import complete!")

#Import Data

wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

source_code = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source_code.content, 'html.parser')
table = soup.find_all('table')       
raw_data = pd.read_html(str(table))

# Transform into Dataframe

data = np.array(raw_data)
df = pd.DataFrame(data[0])

# Naming Columns
df.columns = ['Postcode', 'Borough', 'Neighbourhood']

# Remove the top row
df = df.iloc[1:]


# Remove the top row
df = df[df.Borough != 'Not assigned']
df = df[df.Neighbourhood != 'Not assigned']

df.shape

# df

Import complete!


(211, 3)

### PART 2

In [10]:
import pandas as pd

In [16]:
# import geocoder 

### Importing from CSV

In [11]:
lat_long = pd.read_csv("Geospatial_Coordinates.csv")
lat_long.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Joining the data Sets (df and CSV FILE) 

In [12]:
df_lat_long = pd.merge(left = df, right= lat_long, left_on='Postcode', right_on='Postal Code')

df_lat_long = df_lat_long[["Postcode", "Borough", "Neighbourhood", "Latitude", "Longitude"]]

df_lat_long.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


## PART 3

In [13]:
#Installing Folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: | Killed
Folium installed and imported!


In [28]:
#Importing Others

# from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

# Matplotlib and numpy
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print('Libraries imported.')

Libraries imported.


In [30]:
# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Libraries imported.')

Libraries imported.


## Defining Toronto 

In [31]:
Toronto = df_lat_long['Borough'] == 'Downtown Toronto'

df_Toronto = df_lat_long[Toronto]
df_Toronto= df_Toronto.reset_index()

df_Toronto.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,12,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,13,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,26,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


### Use geopy library to get the latitude and longitude values of Canada

In [36]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


### Creating a map of Canada

In [39]:
map_canada = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto.Latitude, df_Toronto.Longitude, df_Toronto['Borough'], df_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_canada)  
    
map_canada

## Defining FourSquare Credentials

In [41]:
CLIENT_ID = 'O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT' # your Foursquare ID
CLIENT_SECRET = 'FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT
CLIENT_SECRET:FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN


In [44]:
# limit of number of venues returned by Foursquare API
LIMIT = 100 

# define radius
radius = 500 


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT&client_secret=FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN&v=20180605&ll=43.653963,-79.387207&radius=500&limit=100'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c2bdb834c1f677d94ebaaba'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 73,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)




In [48]:
Toronto_venues = getNearbyVenues(names=df_Toronto['Neighbourhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude'])

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [49]:
Toronto_grouped = df_Toronto.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.head()

,Neighbourhood,index,Latitude,Longitude
0,Adelaide,48,43.650571,-79.384568
1,Bathurst Quay,164,43.628947,-79.394420
2,Berczy Park,36,43.644771,-79.373306
3,CN Tower,163,43.628947,-79.394420
4,Cabbagetown,189,43.667967,-79.367675


In [50]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.010000
1,Bathurst Quay,0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,CN Tower,0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,Central Bay Street,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.011765,0.000000,0.000000,0.00,0.011765,0.000000,0.000000
6,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.050000,0.000000,0.040000,0.00,0.010000,0.000000,0.000000
7,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,Church and Wellesley,0.011364,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.011364,0.011364,0.00,0.000000,0.011364,0.000000
9,Commerce Court,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000


## k-means Clustering

In [52]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)

# run k-means clustering
kmeans = KMeans(n_clusters= kclusters, random_state=0).fit(Toronto_grouped_clustering)

In [53]:
Toronto_merged = df_Toronto

# add clustering labels
Toronto_merged['Cluster Labels'] = pd.Series(kmeans.labels_) 

Toronto_not_null = Toronto_merged['Cluster Labels'].notnull()
Toronto_merged = Toronto_merged[Toronto_not_null]
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)

Toronto_merged.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,index,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3
1,3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,1
2,12,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,4
3,13,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,1
4,26,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4


## Map Clusters

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters